In [1]:
import pyspark
from random import randint, random, choice
import matplotlib.pyplot as plt

sc = pyspark.SparkContext('local[*]')

In [9]:
from dataGenerator.rowGenerator import *
from dataGenerator.datasetGenerator import *
from datetime import datetime

dataset = datasetGenerator()
dataset.addGenerator(intGenerator(),min=1,max=1000,keyName="userId")
dataset.addGenerator(intGenerator(),min=1,max=10000,keyName="songId")
dataset.addGenerator(dateGenerator(),startDate=datetime(2021,3,1),keyName="ts")

rdd = sc.parallelize(dataset.generateDataset(1000000)).persist()

In [10]:
rdd = rdd.map(lambda x: (x.get('userId'), x.get('songId'), x.get('ts').date()))
rdd.first()

(697, 8332, datetime.date(2021, 3, 27))

In [12]:
replaysperdayperuser = rdd.map(lambda x: ((x[0], x[2]), 1)).reduceByKey(lambda a,b: a+b)
replaysperdayperuser.top(5, lambda x: x[1])

[((410, datetime.date(2021, 3, 16)), 54),
 ((251, datetime.date(2021, 3, 3)), 53),
 ((483, datetime.date(2021, 4, 4)), 49),
 ((825, datetime.date(2021, 3, 9)), 49),
 ((776, datetime.date(2021, 3, 9)), 48)]

In [18]:
userover30 = replaysperdayperuser.filter(lambda x: x[1] > 30)
userover30.top(5, lambda x: x[1])

[((410, datetime.date(2021, 3, 16)), 54),
 ((251, datetime.date(2021, 3, 3)), 53),
 ((483, datetime.date(2021, 4, 4)), 49),
 ((825, datetime.date(2021, 3, 9)), 49),
 ((776, datetime.date(2021, 3, 9)), 48)]

In [35]:
totLength = userover30.count()
totLength

11398

In [36]:
userover30 = userover30.sortBy(lambda x: 1/x[1])
userover30.take(5)

[((410, datetime.date(2021, 3, 16)), 54),
 ((251, datetime.date(2021, 3, 3)), 53),
 ((483, datetime.date(2021, 4, 4)), 49),
 ((825, datetime.date(2021, 3, 9)), 49),
 ((776, datetime.date(2021, 3, 9)), 48)]